In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


In [2]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(int)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

## Base Para escorar e otimizar

In [4]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Data Table - Otimizacao segmentacao 49.7.csv",
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [5]:
df_lote_escorar=df_lote_102024.copy()

In [6]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [7]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [8]:
df_lote = categorizar(df_lote_escorar)

In [9]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [10]:
df_lote_escorar.shape

(1459, 145)

In [11]:
df_lote = df_lote
#df_lote = df
#df_lote = df.query('chave == 695900014103100001')
P2=(df_lote['premio'])
X1=np.array(df_lote['is_casco_ofer'])
X2=np.array(df_lote['GBM_mercado'])
X3=np.array(df_lote['Premio_IC_100'])
X4=np.array(df_lote['premio_anterior'])
X5=np.array(df_lote['MD_tempo_casa_2'])
X6=np.array(df_lote['per_com'])
X7=np.array(df_lote['per_com_bin_p_1_10'])
X8=np.array(df_lote['canal_bin_Corretor_Mais'])
X9=np.array(df_lote['Cod_End_bin_13509'])
X10=np.array(df_lote['fator_ajuste_bin_f_101_120'])
X11=np.array(df_lote['classe_bonus_bin_00'])
X12=np.array(df_lote['UF_bin_AL_CE_RS'])
X13=np.array(df_lote['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df_lote['UF_bin_GO_MA_MG'])
X15=np.array(df_lote['UF_bin_MT_PR'])
X16=np.array(df_lote['UF_bin_RJ'])
X17=np.array(df_lote['MD_comb_bin_Gasolina'])
X18=np.array(df_lote['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df_lote['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df_lote['fx_ir_12m_bin_01_00_20'])
X21=np.array(df_lote['fx_ir_12m_bin_03_40_60'])
X22=np.array(df_lote['fx_ir_12m_bin_04_60_80'])
X23=np.array(df_lote['fx_ir_12m_bin_05_80_100'])
X24=np.array(df_lote['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df_lote['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df_lote['idade_ult_RNS_bin_06_inf'])
X26=np.array(df_lote['idade_ult_RNS_bin_06_inf'])
def demanda(P2,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P2)  + 1.3647 * np.divide(P2, X1)-0.8782 * np.log(np.divide(P2, X2))-0.795 * np.log(np.divide(P2, X3))-1.6731 * np.log((np.divide(P2, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)



In [12]:
df_lote["Demanda_base"]=demanda(P2,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
df_lote["per_com_pct"] = df_lote['per_com']/100
df_lote["premio_prob"] = df_lote["Demanda_base"]*df_lote["premio"]
df_lote["premio_anterior_prob"] = df_lote["Demanda_base"]*df_lote["premio_anterior"]
df_lote["Act_cost_prob"] = df_lote["Demanda_base"]*df_lote["Act_cost"]                                                                
df_lote["comissao_valor"] = df_lote["per_com_pct"] *df_lote["premio"]
df_lote["comissao_valor_prob"] = df_lote["per_com_pct"] *df_lote["premio_prob"]

In [13]:
df_lote.shape

(1459, 183)

In [14]:
np.mean(df_lote['Demanda_base'])

0.5073676747519267

In [15]:
apolice_ofertada  = np.round(df_lote.shape[0],0)
apolice_estimada  = np.round(np.sum(df_lote["Demanda_base"]),0)
premio_ofertado   = np.mean(df_lote["premio"])
retencao_estimada = np.mean(df_lote["Demanda_base"])
premio_esperado   = np.sum(df_lote["premio_prob"])/np.sum(df_lote["Demanda_base"])
risco_ofertado    = np.mean(df_lote["Act_cost"])
risco_esperado    = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["Demanda_base"])
sp_ofertado       = np.sum(df_lote["Act_cost"])/np.sum(df_lote["premio"])
sp_esperado       = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["premio_prob"])
comis_ofertado    = np.sum(df_lote["comissao_valor"])/np.sum(df_lote["premio"])
comis_esperado    = np.sum(df_lote["comissao_valor_prob"])/np.sum(df_lote["premio_prob"])

premio_anterior   = np.mean(df_lote["premio_anterior"])
premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["Demanda_base"])

ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))
print('premio_anterior: %s' % np.round(premio_anterior,2))
print('premio_anterior_w: %s' % np.round(premio_anterior_w,2))

Apolices Ofertadas: 1459
Apolices Renovadas: 740.0
premio_ofertado: 2993.3
retencao_estimada : 50.74
premio_esperado" : 2388.44
risco_ofertado" : 1726.89
risco_esperado" : 1347.66
sp_ofertado" : 57.69
sp_esperado" : 56.42
ic_ofertado" : 100.78
ic_esperado" : 99.58
premio_anterior: 2863.09
premio_anterior_w: 2634.39


## Otimização Teste 

In [ ]:
#print(df_lote['canal'].value_counts(ascending=False))

In [16]:
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(1459, 183)

In [17]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(1459, 183)

In [18]:
print(np.mean(df_lote_otim['Demanda_basecase'])),
print(np.mean(df_lote_otim['Renewal Demand'])),
print(np.mean(df_lote_otim['Demanda_base']))

0.4969852338386432
0.49692578162505185
0.5073676747519267


#########################################

In [19]:
df_relatorio_bc = df_lote_otim.copy()

df_relatorio_bc["per_com_pct"] = df_relatorio_bc['per_com']/100
df_relatorio_bc["premio_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["premio"]
df_relatorio_bc["Act_cost_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["Act_cost"]                                                                
df_relatorio_bc["comissao_valor"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio"]
df_relatorio_bc["comissao_valor_prob"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_bc.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_bc["Demanda_base"]),0)
premio_ofertado   = np.mean(df_relatorio_bc["premio"])
retencao_estimada = np.mean(df_relatorio_bc["Demanda_base"])
premio_esperado   = np.sum(df_relatorio_bc["premio_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
risco_ofertado    = np.mean(df_relatorio_bc["Act_cost"])
risco_esperado    = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
sp_ofertado       = np.sum(df_relatorio_bc["Act_cost"])/np.sum(df_relatorio_bc["premio"])
sp_esperado       = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_bc["comissao_valor"])/np.sum(df_relatorio_bc["premio"])
comis_esperado    = np.sum(df_relatorio_bc["comissao_valor_prob"])/np.sum(df_relatorio_bc["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

Apolices Ofertadas: 1459
Apolices Renovadas: 740.0
premio_ofertado: 2993.3
retencao_estimada : 50.74
premio_esperado" : 2388.44
risco_ofertado" : 1726.89
risco_esperado" : 1347.66
sp_ofertado" : 57.69
sp_esperado" : 56.42
ic_ofertado" : 100.78
ic_esperado" : 99.58


In [20]:
df = df_lote_otim
P=np.array(df['premio'])
X1=np.array(df_lote['is_casco_ofer'])
X2=np.array(df_lote['GBM_mercado'])
X3=np.array(df_lote['Premio_IC_100'])
X4=np.array(df_lote['premio_anterior'])
X5=np.array(df_lote['MD_tempo_casa_2'])
X6=np.array(df_lote['per_com'])
X7=np.array(df_lote['per_com_bin_p_1_10'])
X8=np.array(df_lote['canal_bin_Corretor_Mais'])
X9=np.array(df_lote['Cod_End_bin_13509'])
X10=np.array(df_lote['fator_ajuste_bin_f_101_120'])
X11=np.array(df_lote['classe_bonus_bin_00'])
X12=np.array(df_lote['UF_bin_AL_CE_RS'])
X13=np.array(df_lote['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df_lote['UF_bin_GO_MA_MG'])
X15=np.array(df_lote['UF_bin_MT_PR'])
X16=np.array(df_lote['UF_bin_RJ'])
X17=np.array(df_lote['MD_comb_bin_Gasolina'])
X18=np.array(df_lote['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df_lote['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df_lote['fx_ir_12m_bin_01_00_20'])
X21=np.array(df_lote['fx_ir_12m_bin_03_40_60'])
X22=np.array(df_lote['fx_ir_12m_bin_04_60_80'])
X23=np.array(df_lote['fx_ir_12m_bin_05_80_100'])
X24=np.array(df_lote['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df_lote['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df_lote['idade_ult_RNS_bin_06_inf'])

# Define a função de demanda
def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


In [21]:
calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26).mean()

0.5073676747519267

In [ ]:
df_lote_otim[['premio',
              'is_casco_ofer',
    'Premio_IC100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf']]

In [83]:
#df = df_lote_otim.iloc[0:500,:]
df = df_lote_otim
df.shape

(1459, 183)

In [84]:
lista_variaveis_na_ordem = [
    'is_casco_ofer',
    'GBM_mercado',
    'Premio_IC100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf']

indicadora=np.array(df.loc[:,lista_variaveis_na_ordem])




In [85]:
df['Demanda_base'].mean()

0.5073676747519267

In [76]:
custo=df['Act_cost'].mean()
custo

1741.1188924550402

In [77]:
df.shape

(500, 183)

In [79]:
import numpy as np
import pandas as pd
import seaborn as sns
from sympy import diff
from sympy import symbols
import sympy as sp
import matplotlib.pyplot as plt

In [86]:
%%time
from scipy.optimize import fsolve
import scipy.optimize as optimize

precos_iniciais=np.array(df['premio'])*1.15
target_demand=df[''].mean()
custo =df['Act_cost']

# Define a função de demanda
def demanda(X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,P):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8 -0.1382 * X9  + 0.3919 * X10 -0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


# Define a função de Lagrange
def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(
                     indicadora[i, 0],
                     indicadora[i, 1], 
                     indicadora[i, 2],
                     indicadora[i, 3],
                     indicadora[i, 4],
                     indicadora[i, 5],
                     indicadora[i, 6],
                     indicadora[i, 7],
                     indicadora[i, 8],
                     indicadora[i, 9],
                     indicadora[i, 10],
                     indicadora[i, 11],
                     indicadora[i, 12],
                     indicadora[i, 13],
                     indicadora[i, 14],
                     indicadora[i, 15],
                     indicadora[i, 16],
                     indicadora[i, 17],
                     indicadora[i, 18],
                     indicadora[i, 19],
                     indicadora[i, 20],
                     indicadora[i, 21],
                     indicadora[i, 22],
                     indicadora[i, 23],
                     indicadora[i, 24],
                     indicadora[i, 25],P[i])
        objetivo += D_i * (P[i] - custo[i])
        soma_demandas += D_i
    
    # Definindo a função de Lagrange com a restrição
    return objetivo - L * (soma_demandas - target_demand*df.shape[0])

# Aplicando a diferenciação numérica
def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-3  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
    return dLambda

# Definindo a função para otimizar os preços
def otimizar_precos(indicadora, precos_iniciais):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
    # Calculando os preços otimizados
    X_otimizado = optimize.fsolve(dfunc, X_inicial, args=(indicadora),xtol=1e-06, maxfev=100)
    
    # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    precos_otimizados = X_otimizado[:-1]
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida

precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("λ:", lambida)
#print("Preços Otimizados:", precos_otimizados[0], precos_otimizados[1])
print("Preços Otimizados:",precos_otimizados[0:20,:])
#print("P1:", precos_otimizados[0])
#print("P2:", precos_otimizados[1])

KeyboardInterrupt: 

In [70]:
df2=df
df2['precos_otimizados']=precos_otimizados
df2[['premio','precos_otimizados']]

,premio,precos_otimizados
0,2283.285524,2739.942629
1,2868.227713,3441.873256
2,4664.302380,5597.162856
3,4305.597750,5166.717300
4,2655.196079,3186.235295
...,...,...
95,2535.876000,3043.051200
96,2233.263974,2679.916768
97,1721.496000,2065.795200
98,2334.520333,2801.424399


In [71]:
P=np.array(df2['premio'])
X1=np.array(df2['is_casco_ofer'])
X2=np.array(df2['GBM_mercado'])
X3=np.array(df2['Premio_IC_100'])
X4=np.array(df2['premio_anterior'])
X5=np.array(df2['MD_tempo_casa_2'])
X6=np.array(df2['per_com'])
X7=np.array(df2['per_com_bin_p_1_10'])
X8=np.array(df2['canal_bin_Corretor_Mais'])
X9=np.array(df2['Cod_End_bin_13509'])
X10=np.array(df2['fator_ajuste_bin_f_101_120'])
X11=np.array(df2['classe_bonus_bin_00'])
X12=np.array(df2['UF_bin_AL_CE_RS'])
X13=np.array(df2['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df2['UF_bin_GO_MA_MG'])
X15=np.array(df2['UF_bin_MT_PR'])
X16=np.array(df2['UF_bin_RJ'])
X17=np.array(df2['MD_comb_bin_Gasolina'])
X18=np.array(df2['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df2['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df2['fx_ir_12m_bin_01_00_20'])
X21=np.array(df2['fx_ir_12m_bin_03_40_60'])
X22=np.array(df2['fx_ir_12m_bin_04_60_80'])
X23=np.array(df2['fx_ir_12m_bin_05_80_100'])
X24=np.array(df2['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df2['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df2['idade_ult_RNS_bin_06_inf'])

# Define a função de demanda
def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


In [72]:
calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26).mean()

0.5306874932695371

In [ ]:

# Verificar os preços otimizados e a margem esperada otimizada 
df['precos_otimizados'] = precos_otimizados
df['retencao_otimizada'] = demanda(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
df['retencao_base_esperado']=demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
df['margem_esperada_otimizada'] =   df['retencao_otimizada']*(df['precos_otimizados'] - df['Act_cost'])

df['margem_esperada_base_case'] = df['retencao_base_esperado']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()
margem_esperada_total_otimizada=df['margem_esperada_otimizada'].sum()

# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Base Case:", df['retencao_base_esperado'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])

In [ ]:
#df[['premio','retencao_base_esperado','preco_otimizado','retencao_otimizada']]

In [ ]:
#df#.to_csv(r'C:/Users/vcatelan/Documents/Scripts/dftocsv2.csv', sep='|', encoding='utf-8', header='true')
#df.to_excel(r'df.xlsx', sep='|', encoding='utf-8', header='true')

#criando a função

In [ ]:
df_relatorio_otim = df.copy()

df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

In [ ]:
df.to_csv(r'df_total.csv', sep='|' ,header='true')

In [ ]:
df_relatorio_otim[['Demanda_base','retencao_otimizada','premio','preco_otimizado']]


In [ ]:
prices_df = pd.DataFrame({
    'chave': df_relatorio["chave"],
    'Act_cost':  df_relatorio["Act_cost"],
    'Premio ofertado com batente':  df_relatorio["Premio_oferecido_com_batente"],
    'Optimized Prices': df_relatorio["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_relatorio["prob_est"],
    'renewal demand':df_relatorio["demanda otimizada"],
    'variação':df_relatorio["premio"]/df_relatorio["Premio_oferecido_com_batente"]
})
prices_df.head(20)

########

In [ ]:
%%time
target_demand=0.48

df = df_lote_otim
P=(df['premio'])
X1=np.array(df['is_casco_ofer'])
X2=np.array(df['Premio_IC100'])
X3=np.array(df['premio_anterior'])
X4=np.array(df['MD_tempo_casa_2'])
X5=np.array(df['per_com'])
X6=np.array(df['per_com_bin_p_1_10'])
X7=np.array(df['canal_bin_Corretor_Mais'])
X8=np.array(df['Cod_End_bin_13509'])
X9=np.array(df['fator_ajuste_bin_f_101_120'])
X10=np.array(df['classe_bonus_bin_00'])
X11=np.array(df['UF_bin_AL_CE_RS'])
X12=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X13=np.array(df['UF_bin_GO_MA_MG'])
X14=np.array(df['UF_bin_MT_PR'])
X15=np.array(df['UF_bin_RJ'])
X16=np.array(df['MD_comb_bin_Gasolina'])
X17=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X18=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X19=np.array(df['fx_ir_12m_bin_01_00_20'])
X20=np.array(df['fx_ir_12m_bin_03_40_60'])
X21=np.array(df['fx_ir_12m_bin_04_60_80'])
X22=np.array(df['fx_ir_12m_bin_05_80_100'])
X23=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X24=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X25=np.array(df['idade_ult_RNS_bin_06_inf'])


# Função de demanda para calcular a retenção com base no preço 
def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25):
    D = np.exp(3.3458-0.4638 * np.log(P)-5.1803 * np.divide(P, X1)-1.0904 * np.log(np.divide(P, X2))-1.2646 * np.log((np.divide(P, X3)))  + 0.228 * np.log(X4 + 1)  + 0.0137 * X5  + 0.1566 * X6  + 0.0576 * X7-0.2318 * X8  + 0.16 * X9-0.2332 * X10-0.2319 * X11-0.2498 * X12-0.0697 * X13-0.1227 * X14  + 0.2475 * X14  + 0.3917 * X16-0.4536 * X17-0.1999 * X18-0.7509 * X19  + 0.4693 * X20  + 1.1327 * X21  + 2.0154 * X22-0.1554 * X23-0.5014 * X24-0.4664 * X25)/(1 + np.exp(3.3458-0.4638 * np.log(P)-5.1803 * np.divide(P, X1)-1.0904 * np.log(np.divide(P, X2))-1.2646 * np.log((np.divide(P, X3)))  + 0.228 * np.log(X4 + 1)  + 0.0137 * X5  + 0.1566 * X6  + 0.0576 * X7-0.2318 * X8  + 0.16 * X9-0.2332 * X10-0.2319 * X11-0.2498 * X12-0.0697 * X13-0.1227 * X14  + 0.2475 * X14  + 0.3917 * X16-0.4536 * X17-0.1999 * X18-0.7509 * X19  + 0.4693 * X20  + 1.1327 * X21  + 2.0154 * X22-0.1554 * X23-0.5014 * X24-0.4664 * X25) )  
    return D

df['retencao_base_esperado']=calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)

df['retencao_target']=target_demand

fator_retencao = np.mean(df['retencao_base_esperado'])/np.mean(df['retencao_target'])

# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(precos):
    df['precos']=precos
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada'] = calcular_retencao(precos,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
    df['margem_esperada_otimizada'] =   df['retencao_otimizada']*(df['precos'] - df['Act_cost'])
    
    # Calcular a margem esperada total (somatório)
    margem_esperada_total = df['margem_esperada_otimizada'].sum()

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -margem_esperada_total

# Definir a restrição: a retenção média deve ser igual à retenção base média 
def restricao(precos):
    df['precos']=precos
    
    # Calcular a retenção otimizada média com os preços dados
    retencao_otimizada_soma = calcular_retencao(precos,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25).sum()
    
    return np.round(retencao_otimizada_soma,3) - np.round(df['retencao_target'].sum,3)

# Estimativas iniciais (os preços base)

#precos_iniciais = fator_retencao*df['premio'].values

premio_base = df['premio'].values

precos_iniciais = 1*df['premio'].values

# Definir os limites para os preços (0.9 * preco_base <= preco_otimizado <= 1.1 * preco_base) 
limites = [(0.9 * p, 1.1 * p) for p in premio_base]

# Definir as restrições no formato exigido pelo método 'minimize'
restricoes = ({'type': 'eq', 'fun': restricao})

# Executar a otimização 

solution = minimize(objetivo,
                    precos_iniciais,
                    method='L-BFGS-B',
                    bounds=limites,
                    constraints=restricoes) 
# Resultados
precos_otimizados = solution.x
margem_esperada_total_otimizada = -resultado.fun

# Verificar os preços otimizados e a margem esperada otimizada 
df['preco_otimizado'] = solution.x 
df['retencao_otimizada'] = calcular_retencao(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)

df['margem_esperada_base_case'] = df['retencao_base_esperado']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()

# Exibir os resultados
print(solution.message) 
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Base Case:", df['retencao_base_esperado'].mean())
print("Premio Medio Otimizado:", df['preco_otimizado'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])


In [ ]:
from handythread import foreach

In [ ]:
!pip install pp

In [92]:
import numpy as np
import sympy as sp
from scipy import optimize


C = 1000
# Definindo as variáveis
P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, C, Lambda, target= sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26 C Lambda target')

# Definindo a função F
F = sp.exp(1.3356 - 0.2576 * sp.log(P) + 1.3647 * (P / X1) - 0.8782 * sp.log(P / X2) - 0.795 * sp.log(P / X3) 
           - 1.6731 * sp.log(P / X4) + 0.3797 * sp.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 
           - 0.1382 * X9 + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 - 0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 
           + 0.2817 * X16 + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 + 0.4343 * X21 + 1.1066 * X22 
           + 1.9561 * X23 - 0.0742 * X24 - 0.8459 * X25 - 0.7094 * X26) * (P - C) - Lambda * (sp.exp(1.3356 - 0.2576 * sp.log(P) + 1.3647 * (P / X1) - 0.8782 * sp.log(P / X2) - 0.795 * sp.log(P / X3) 
             - 1.6731 * sp.log(P / X4) + 0.3797 * sp.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 - 0.1382 * X9 
             + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 - 0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 + 0.2817 * X16 
             + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 + 0.4343 * X21 + 1.1066 * X22 + 1.9561 * X23 - 0.0742 * X24 
                - 0.8459 * X25 - 0.7094 * X26) - target)

# Derivando F em relação a P e Lambda
dF_dP = sp.diff(F, P)
dF_dLambda = sp.diff(F, Lambda)

# Convertendo as derivadas em funções utilizáveis
dF_dP_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, C, Lambda, target), dF_dP, 'numpy')
dF_dLambda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, C, Lambda, target), dF_dLambda, 'numpy')

# Aplicando as derivadas calculadas
def dfunc(X, indicadora, target):
    P_vals = X[:-1]
    L_val = X[-1]
    dLambda = np.zeros(len(X))
    
    for i in range(len(P_vals)):
        dLambda[i] = dF_dP_func(P_vals[i], *indicadora[i], L_val, target)
    
    dLambda[-1] = dF_dLambda_func(P_vals[0], *indicadora[0], L_val, target)  # Usando o primeiro conjunto de indicadores para Lambda
    
    return dLambda

# Definindo a função para otimizar os preços
def otimizar_precos(indicadora, precos_iniciais, target):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
    # Calculando os preços otimizados
    X_otimizado = optimize.fsolve(dfunc, X_inicial, args=(indicadora, target), xtol=1e-06, maxfev=500)
    
    # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    precos_otimizados = X_otimizado[:-1]
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida

# Exemplo de uso
indicadora = np.random.rand(10, 26)  # Exemplo de matriz de indicadores
precos_iniciais = np.random.rand(10) * 100  # Exemplo de preços iniciais
target = 1000  # Exemplo de demanda média

precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais, target)
print("λ:", lambida)
print("Preços Otimizados:", precos_otimizados)

TypeError: _lambdifygenerated() missing 1 required positional argument: 'target'